# Wisconsin faculty salaries

In this demo, we'll analyze data on average faculty salaries in Wisconsin, as compiled by the [Chronicle of Higher Education](http://data.chronicle.com/category/state/Wisconsin/faculty-salaries/).  The file ```wisconsinfaculty.txt``` contains data on faculty salaries at the 25 highest-paying institutions in Wisconsin in the 2013-2014 academic year, ordered by pay for faculty with the rank of Professor.

In [1]:
import pandas as pd
import numpy as np

In [13]:
# To start, we look at the first 10 lines of the file, to understand its structure.

with open("wisconsinfaculty.txt") as wisc_file:
    line_num = 0
    for line in wisc_file:
        print("Line number:", line_num, line)
        line_num +=1
        if line_num==10:
            break
        

Line number: 0 Institution	Professors	Associate Professors	Assistant Professors	Instructors	Lecturers	Unranked

Line number: 1 Marquette University

Line number: 2 Wisconsin | 4-year private | Doctoral	$118,953	$83,853	$72,702	$55,863	$53,937	$56,052

Line number: 3 University of Wisconsin at Madison

Line number: 4 Wisconsin | 4-year public | Very high research activity	$113,094	$84,510	$77,157	$51,120	$66,078	$34,407

Line number: 5 University of Wisconsin at Milwaukee

Line number: 6 Wisconsin | 4-year public | High research activity	$97,317	$74,925	$70,173	$49,077	$45,513	--

Line number: 7 Lawrence University

Line number: 8 Wisconsin | 4-year private | Arts and sciences baccalaureate	$90,324	$69,597	$54,396	$50,859	--	--

Line number: 9 Carthage College



We see that after a header line, institution names are on odd lines and institution data is on even lines.

We want to build a data frame with the following headers: Institution, Is Public, Professors, Associate Professors	Assistant Professors, Instructors, Lecturers, and Unranked.

Let's start by making sure we can extract the appropriate information.  We'll test our code on the first few lines of the file, to make sure it works.

In [3]:
%%time
with open("wisconsinfaculty.txt") as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    line_num=1
    while line_num < 10:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        print(inst, is_public, salary_list)
            
        line_num += 2

Marquette University False [118953, 83853, 72702, 55863, 53937, 56052]
University of Wisconsin at Madison True [113094, 84510, 77157, 51120, 66078, 34407]
University of Wisconsin at Milwaukee True [97317, 74925, 70173, 49077, 45513, nan]
Lawrence University False [90324, 69597, 54396, 50859, nan, nan]
Carthage College False [84501, 68958, 57816, 34128, nan, nan]
Wall time: 1e+03 µs


Now we have to insert our information in a data frame.  A naive way to do this is to create an empty data frame and append the information for each row as we compute it.

In [5]:
%%time 
with open("wisconsinfaculty.txt") as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    # create an empty data frame with the appropriate columns
    column_names = ('Institution', 'Is Public', 'Professors', 'Associate Professors', 'Assistant Professors', 'Instructors', 'Lecturers', 'Unranked')
    fac_df = pd.DataFrame(columns=column_names)
    
    line_num=1
    while line_num < 10:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        fac_df = fac_df.append(pd.DataFrame([[inst, is_public]+salary_list], columns = column_names))
            
        line_num += 2

print(fac_df)

                            Institution Is Public  Professors  \
0                  Marquette University     False    118953.0   
0    University of Wisconsin at Madison      True    113094.0   
0  University of Wisconsin at Milwaukee      True     97317.0   
0                   Lawrence University     False     90324.0   
0                      Carthage College     False     84501.0   

   Associate Professors  Assistant Professors  Instructors  Lecturers  \
0               83853.0               72702.0      55863.0    53937.0   
0               84510.0               77157.0      51120.0    66078.0   
0               74925.0               70173.0      49077.0    45513.0   
0               69597.0               54396.0      50859.0        NaN   
0               68958.0               57816.0      34128.0        NaN   

   Unranked  
0   56052.0  
0   34407.0  
0       NaN  
0       NaN  
0       NaN  
Wall time: 16 ms


We see that appending the information for each row is significantly slower than merely computing the information itself.  The problem is that appending data frames is a memory-intensive operation.

We can make our code faster by storing the information we compute in a list of dictionaries, and converting to a data frame at the end.

In [8]:
%%time
with open("wisconsinfaculty.txt") as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    dict_list = []
    column_names = ('Institution', 'Is Public', 'Professors', 'Associate Professors', 'Assistant Professors', 'Instructors', 'Lecturers', 'Unranked')
    
    line_num=1
    while line_num < 10:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
#         print(pipe_list[1])
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
#         print(pipe_list[2])
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        dict_list.append(dict(zip(column_names, [inst,is_public]+salary_list)))
            
        line_num += 2

fac_df = pd.DataFrame(dict_list)
print(fac_df)

   Assistant Professors  Associate Professors  \
0                 72702                 83853   
1                 77157                 84510   
2                 70173                 74925   
3                 54396                 69597   
4                 57816                 68958   

                            Institution  Instructors Is Public  Lecturers  \
0                  Marquette University        55863     False    53937.0   
1    University of Wisconsin at Madison        51120      True    66078.0   
2  University of Wisconsin at Milwaukee        49077      True    45513.0   
3                   Lawrence University        50859     False        NaN   
4                      Carthage College        34128     False        NaN   

   Professors  Unranked  
0      118953   56052.0  
1      113094   34407.0  
2       97317       NaN  
3       90324       NaN  
4       84501       NaN  
Wall time: 6 ms


We're ready to run our code on the full file!

In [9]:
with open("wisconsinfaculty.txt") as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    dict_list = []
    column_names = ('Institution', 'Is Public', 'Professors', 'Associate Professors', 'Assistant Professors', 'Instructors', 'Lecturers', 'Unranked')
    
    #We rely on our break command at the end of the file to exit the loop gracefully
    
    while True:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        dict_list.append(dict(zip(column_names, [inst,is_public]+salary_list)))

fac_df = pd.DataFrame(dict_list)
fac_df

,Assistant Professors,Associate Professors,Institution,Instructors,Is Public,Lecturers,Professors,Unranked
0,72702,83853,Marquette University,55863.0,False,53937.0,118953,56052.0
1,77157,84510,University of Wisconsin at Madison,51120.0,True,66078.0,113094,34407.0
2,70173,74925,University of Wisconsin at Milwaukee,49077.0,True,45513.0,97317,NaN
3,54396,69597,Lawrence University,50859.0,False,NaN,90324,NaN
4,57816,68958,Carthage College,34128.0,False,NaN,84501,NaN
5,69642,73674,Milwaukee School of Engineering,54603.0,False,NaN,83034,NaN
6,54828,65718,Carroll University,NaN,False,51273.0,81315,NaN
7,52956,69543,Saint Norbert College,33822.0,False,NaN,81018,47052.0
8,65610,64980,University of Wisconsin at Whitewater,NaN,True,48186.0,78966,NaN
9,57789,62649,University of Wisconsin at La Crosse,NaN,True,42588.0,78561,NaN


Now that we have created the data frame, it's easy to extract summary statistics!

In [11]:
fac_df.describe()

C:\Users\dmarkham\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Assistant Professors,Associate Professors,Instructors,Lecturers,Professors,Unranked
count,25.000000,25.00000,15.000000,15.000000,25.000000,3.000000
mean,58452.480000,65414.16000,48934.200000,46088.400000,79972.920000,45837.000000
std,7884.770733,7795.29043,13159.404303,6928.117481,12950.308699,10873.530935
min,47133.000000,56475.00000,26856.000000,37755.000000,67860.000000,34407.000000
25%,52956.000000,59121.00000,NaN,NaN,71199.000000,NaN
50%,56439.000000,62649.00000,NaN,NaN,78048.000000,NaN
75%,59526.000000,69543.00000,NaN,NaN,81315.000000,NaN
max,77157.000000,84510.00000,83124.000000,66078.000000,118953.000000,56052.000000


In [12]:
fac_df.groupby('Is Public').describe()

C:\Users\dmarkham\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


Assistant Professors  Associate Professors   Instructors  \
Is Public                                                                   
False     count             13.000000             13.000000     10.000000   
          mean           56830.846154          66135.461538  45580.500000   
          std             8462.767857           8028.157682  11222.743983   
          min            47133.000000          56538.000000  26856.000000   
          25%            51606.000000          59121.000000           NaN   
          50%            53388.000000          65718.000000           NaN   
          75%            57816.000000          69597.000000           NaN   
          max            72702.000000          83853.000000  61902.000000   
True      count             12.000000             12.000000      5.000000   
          mean           60209.250000          64632.750000  55641.600000   
          std             7144.154688           7809.613390  15438.668573   
          min            52839.000000          56475.000000  47151.000000   
          25%            56229.750000          60630.750000           NaN   
          50%            57528.000000          62455.500000           NaN   
          75%            61047.000000          65178.000000           NaN   
          max            77157.000000          84510.000000  83124.000000   

                    Lecturers     Professors      Unranked  
Is Public                                                   
False     count      3.000000      13.000000      2.000000  
          mean   47655.000000   81182.076923  51552.000000  
          std     8676.504135   13023.710150   6363.961031  
          min    37755.000000   67860.000000  47052.000000  
          25%             NaN   72927.000000           NaN  
          50%             NaN   78525.000000           NaN  
          75%             NaN   83034.000000           NaN  
          max    53937.000000  118953.000000  56052.000000  
True      count     12.000000      12.000000      1.000000  
          mean   45696.750000   78663.000000  34407.000000  
          std     6823.865435   13314.551527           NaN  
          min    40104.000000   68148.000000  34407.000000  
          25%    42302.250000   71007.750000           NaN  
          50%    44383.500000   74722.500000           NaN  
          75%    45686.250000   78662.250000           NaN  
          max    66078.000000  113094.000000  34407.000000